# genetic-nn-tutorial
Basic tutorial for training feed-forward neural networks with genetic algorithms

This code uses a genetic algorithm to train a feed forward neural network to learn to approximate the cosine function.

The only (necessary) dependencies are basic: numpy, random and math.

Note that this is written and published only with education purposes in mind, real-world training of neural networks is generally much more efficient with backpropagation.

Genetic algorithms are worth understanding because, while being time-consuming, they make no requirements for end-to-end differentiability, can handle discrete values (although not done here)  and are easy to code.

In [ ]:
# Note that we only have very basic dependencies
import random
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
# Our dataset
dataset = []

# Where to start the X's
x_runner = -4.0
increment = 0.04

# Generate the dataset, we're going to learn the cos(x) function
for i in range(200):
    dataset.append([x_runner, math.cos(x_runner)])
    x_runner += increment

In [ ]:
# Plot the whole dataset
fig = plt.figure(figsize=(12, 8))
plt.plot(list(item[0] for item in dataset), list(item[1] for item in dataset))
fig.suptitle('Cosine function')
plt.xlabel('x', fontsize=18)
plt.ylabel('cos(x)', fontsize=16)
plt.show()


In [ ]:
# Split into training and testing sets
training_dataset = dataset[:150]
testing_dataset = dataset[-50:]
# Training inputs
x_training = list(item[0] for item in training_dataset)
# Training outputs
y_training = list(item[1] for item in training_dataset)
# Testing inputs
x_testing = list(item[0] for item in testing_dataset)
# Testing outputs
y_testing = list(item[1] for item in testing_dataset)

# Plot training and testing datasets
fig = plt.figure(figsize=(12, 8))
fig.suptitle('Training and testing data')
plt.plot(x_training, y_training, color='green')
plt.plot(x_testing, y_testing, color='blue')
plt.xlabel('x', fontsize=18)
plt.ylabel('cos(x)', fontsize=16)
plt.show()


In [ ]:
# Shuffle the dataset to make the training and testing sets more evenly distributed
random.shuffle(dataset)

# Redo the split
# Split into training and testing sets
training_dataset = dataset[:150]
testing_dataset = dataset[-50:]
# Training inputs
x_training = list(item[0] for item in training_dataset)
# Training outputs
y_training = list(item[1] for item in training_dataset)
# Testing inputs
x_testing = list(item[0] for item in testing_dataset)
# Testing outputs
y_testing = list(item[1] for item in testing_dataset)

# Normalize X values
for i in range(len(x_training)):
    x_training[i] = x_training[i] / (200*increment - 100*increment)

for i in range(len(x_testing)):
    x_testing[i] = x_testing[i] / (200*increment - 100*increment)


# Plot training and testing datasets
fig = plt.figure(figsize=(12, 8))
fig.suptitle('Training and testing data (randomized)')
plt.scatter(x_training, y_training, color='green')
plt.scatter(x_testing, y_testing, color='blue')
plt.xlabel('x', fontsize=18)
plt.ylabel('cos(x)', fontsize=16)
plt.show()


In [ ]:
## Since we're going to be using a genetic algorithm, we need something that will give us a randomly initialized neural network with the structure we want. We will assume one hidden layer.
def produce_neural_network(inputs_size, hidden_layer_size, outputs_size, randomize = True):
    net = {}
    net['inputs_size'] = inputs_size
    net['hidden_size'] = hidden_layer_size
    net['outputs_size'] = outputs_size
    if randomize:
        net['weights_in'] = np.subtract(np.random.rand(inputs_size*hidden_layer_size)*5, 2.5)
        net['weights_out'] = np.subtract(np.random.rand(hidden_layer_size * outputs_size)*5, 2.5)
    net['biases'] = np.subtract(np.random.rand(hidden_layer_size)*5, 2.5)
    return net

In [ ]:
# And we need a way to run that neural network on inputs to generate outputs
def run_neural_network(x, net):
    if len(x) != net['inputs_size']:
        raise Exception('Input dimensions do not match network structure')    
    hidden_layer = np.add(np.multiply(x, net['weights_in']), net['biases'])
    # Now for RELU activation
    hidden_layer = hidden_layer * (hidden_layer > 0)
    output_layer = np.matmul( np.matrix([hidden_layer]), np.matrix([net['weights_out']]).T)
    return output_layer

In [ ]:
# Genetic algorithm function: As we'll see in a little bit, we need a way to "breed" arrays
def breed_arrays(mother, father, crossover, mutation):
    if len(mother) != len(father):
        raise Exception('Incompatible parents')
    child = []
    # Both parents have the same genome size and the child will have the same
    genome_size = len(mother)
    cutoff = random.randint(1, genome_size-1)
    genome_index = 0
    while len(child) < genome_size:
        if genome_index < cutoff:
            child.append(mother[genome_index])
        else:
            child.append(father[genome_index])
        mutation_decider = random.random()
        if mutation_decider < mutation:
            child[genome_index] += random.random() * random.random() * (2.0 * random.random() - 1.0)
        genome_index += 1
    return child

In [ ]:
# Genetic algorithm function: We need to breed neural networks, creating one new network from two parents
def breed_neural_networks(mother_net, father_net, crossover = 0.6, mutation = 0.005 ):
    if mother_net['inputs_size'] != father_net['inputs_size'] or mother_net['hidden_size'] != father_net['hidden_size'] or  mother_net['outputs_size'] != father_net['outputs_size']:
        raise Exception('Incompatible parents')
    child_net = produce_neural_network(mother_net['inputs_size'], mother_net['hidden_size'], mother_net['outputs_size'], randomize=False)
    child_net['biases'] = breed_arrays(mother_net['biases'], father_net['biases'], crossover, mutation)
    child_net['weights_in'] = breed_arrays(mother_net['weights_in'], father_net['weights_in'], crossover, mutation)
    child_net['weights_out'] = breed_arrays(mother_net['weights_out'], father_net['weights_out'], crossover, mutation)
    return child_net

In [ ]:
# The final part of the genetic stuff is a fitness function
def evaluate_neural_network(net, input_data, output_data):
    if len(input_data) != len(output_data):
        raise Exception('Input and output data arrays have different lengths')
    net['input_output'] = []
    net_input = []
    net_output = []
    sumSquaredError = 0
    for dataIndex in range(len(input_data)):
        output = run_neural_network([ input_data[dataIndex] ], net)
        net_input.append(input_data[dataIndex])
        net_output.append(output)
        sampleError = abs(output - output_data[dataIndex])
        sumSquaredError += sampleError * sampleError
    net['input_output'].append(net_input)
    net['input_output'].append(net_output)   
    # We return the sum squared error
    return sumSquaredError

In [ ]:
# Need to set some parameters now

# Number of networks in each generation
generation_size = 40
# Size of the hidden layer
hidden_layer_size = 40
# Maximum number of generations to cycle through
generations_max = 2000
# Acceptable error threshold
acceptable_error = 0.001


In [ ]:
# Let's go ahead and create the first generation
current_generation = []
sorted_by_fitness = []
for i in range(generation_size):
    current_generation.append(produce_neural_network(1, hidden_layer_size, 1))

In [ ]:
from IPython.display import clear_output

print('Starting evolution')

best_score_per_generation = []

# Our main loop that does the evolution
for j in range(generations_max):
    # Evaluate the current generation
    for k in range(generation_size):
        current_generation[k]['sse'] = evaluate_neural_network(current_generation[k], x_training, y_training)
    # Sort by fitness
    sorted_by_fitness = sorted(current_generation, key=lambda k: k['sse'])

    # Print the best one
    print('Best solution in generation', j, 'has error', sorted_by_fitness[0]['sse'].item(0))

    if j == generations_max-1 or (len(best_score_per_generation) > 0 and best_score_per_generation[-1] > sorted_by_fitness[0]['sse']):
        clear_output(wait=True)
        print('Best solution in generation', j, 'has error', sorted_by_fitness[0]['sse'].item(0))
        if j == generations_max-1:
            print('Evolution has finished')
        # Plot function currently learned
        fig = plt.figure(figsize=(12, 8))
        plt.scatter(x_training, y_training, color='green')
        plt.scatter(sorted_by_fitness[0]['input_output'][0], sorted_by_fitness[0]['input_output'][1], color='red')
        fig.suptitle('Training data versus learned function')
        plt.ylabel('cos(x)')
        plt.xlabel('x normalized')
        plt.show()
    else:
        print('Best solution in generation', j, 'has error', sorted_by_fitness[0]['sse'].item(0))
        
    best_score_per_generation.append(sorted_by_fitness[0]['sse'].item(0))

    # Stop if we're close enough already
    if sorted_by_fitness[0]['sse'] < acceptable_error:
        print('Done!')
        break

    # Create a new generation and make sure we hang onto the best solution so far
    new_generation = []
    new_generation.append(sorted_by_fitness[0])
    # Do breeding to create the next generation
    for l in range (1, generation_size):
        # Randomly pick mothers and fathers
        mother_index = 0
        father_index = 0
        while mother_index == father_index:
            mother_index = int(round((generation_size-1) * (random.random() * random.random())))
            father_index = int(round((generation_size-1) * random.random()))
        child = breed_neural_networks(sorted_by_fitness[mother_index], sorted_by_fitness[father_index])
        new_generation.append(child)
    current_generation = new_generation
print('Evolution done')


In [ ]:
print('Starting test phase')

actual_output = []
# Let's test what we learned on the testing data
for i in range(len(x_testing)):
    input_value = x_testing[i]
    desired_output_value = y_testing[i]
    print("input value " + str(input_value) + " output value " + str(desired_output_value))
    result = run_neural_network([input_value], sorted_by_fitness[0])
    actual_output.append(result)
    print('From X=',input_value,'we wanted',desired_output_value,'and got',actual_output[i].item(0))

In [ ]:
# Plot learning progress
fig = plt.figure(figsize=(12, 8))
fig.suptitle('Results')
plt.plot(best_score_per_generation)
plt.xlabel('iteration', fontsize=18)
plt.ylabel('error', fontsize=16)
plt.show()

In [ ]:
# Plot results vs "ground truth"
fig = plt.figure(figsize=(12, 8))

plt.scatter(x_training, y_training, color='green')
plt.scatter(x_testing, y_testing, color='blue')
plt.scatter(x_testing, actual_output, color='red')
fig.suptitle('Training, testing and result data')
plt.ylabel('cos(x)')
plt.xlabel('x normalized')
plt.show()
